In [21]:
from importlib import reload
import lancedb
import pandas as pd
from dotenv import load_dotenv

load_dotenv("/Users/leon/.env")

# DB specifications
LANCEDB_DIR = "/Users/leon/Documents/study/MA/lancedb"
TABLE_NAME_DOCS = "documents"
TABLE_NAME_CHUNKS = "chunks"

db = lancedb.connect(LANCEDB_DIR)

In [10]:
def add_fictional_creation_date(row):
    if str(row["doc_id"]).startswith("400"):
        return pd.to_datetime("2010-01-01")
    if str(row["doc_id"]).startswith("300"):
        return pd.to_datetime("2015-01-01")
    if str(row["doc_id"]).startswith("100"):
        return pd.to_datetime("2005-01-01")
    return pd.to_datetime("2000-01-01")

In [22]:
from utils import io_helpers

reload(io_helpers)

documents = io_helpers.get_documents(read_embeddings=True).drop(columns="original_doc_ids")
documents["creation_date"] = documents.apply(add_fictional_creation_date, axis=1)

table = db.create_table(TABLE_NAME_DOCS, data=documents, exist_ok=True)
db[TABLE_NAME_DOCS].head(1)

pyarrow.Table
doc_id: int64
domain: string
content: string
embedding: list<item: double>
  child 0, item: double
creation_date: timestamp[ns]
----
doc_id: [[40]]
domain: [["Finance"]]
content: [["Acme Government Solutions is a government industry company established on June 1, 2001 in Washington, D.C., specializing in providing comprehensive government services and solutions.
In January 2021, Acme Government Solutions made a significant decision to distribute $5 million of dividends to its shareholders. This move not only enhanced shareholder returns but also showcased the company's commitment to rewarding its investors. This dividend distribution was a result of the company's successful acquisition of a major government contract worth $100 million in March 2021. This acquisition expanded Acme Government Solutions' service portfolio and increased its revenue potential. Furthermore, in April 2021, the company announced plans to establish regional offices in three new states, thereby exp

In [23]:
from langchain_community.document_loaders import DataFrameLoader

table_data: pd.DataFrame = table.search().to_pandas()

loader = DataFrameLoader(table_data, page_content_column="content")
lc_documents = loader.load()

In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

lc_chunks = RecursiveCharacterTextSplitter(
    ["\n\n", "\n", " ", ""], keep_separator=False, chunk_size=1024, chunk_overlap=128
).split_documents(lc_documents)

print(len(lc_chunks))
lc_chunks[-5:]

1727


[Document(metadata={'doc_id': 400059, 'domain': 'Finance', 'embedding': array([ 0.03652241,  0.02207698,  0.02154485, ..., -0.00806634,
         0.03737901,  0.01314106]), 'creation_date': Timestamp('2010-01-01 00:00:00')}, page_content="In April 2020, Retail Emporium underwent a noteworthy asset restructuring, enhancing operational efficiency and company value. This was followed by the acquisition of a 40% stake in Fancy Apparel Company in May, augmenting market share and control. July saw the acquisition of HomeGoods Corporation’s assets at a total value of $100 million, broadening business scope and competitiveness. In August, a crucial debt restructuring reduced liabilities by $50 million, and a large-scale financing activity in September raised $200 million, fortifying financial strength for expansion.\n\nThe company's financial indicators for 2020 were strong, with an operating income of $550 million and a net profit of $50 million, driven by various strategic moves and operation

In [ ]:
from langchain.vectorstores import LanceDB
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
from lancedb.embeddings import get_registry
from lancedb.pydantic import LanceModel, Vector

from tenacity import (
    retry,
    wait_random_exponential,
    retry_if_exception_type,
    stop_after_attempt,
)